In [105]:
from typing import Annotated, Sequence, TypedDict, List, Dict
from dotenv import load_dotenv  
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, ToolMessage, SystemMessage
from langchain_groq import ChatGroq
from langchain_core.tools import tool
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, END, START
from langgraph.prebuilt import ToolNode
import os

In [106]:
load_dotenv()
GROQ_API_KEY=os.getenv("GROQ_API_KEY")
os.environ["GROQ_API_KEY"]= GROQ_API_KEY

In [107]:
llm = ChatGroq(model="llama-3.1-8b-instant")
# llm.invoke("hello who are you").content

In [108]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    user_prompt: str                                 # Original user input prompt
    scraped_data: List[Dict[str, str]]               # Raw web content scraped
    subtopics: List[str]                             # List of 5–7 subtopics generated by LLM
    full_script: List[Dict[str, str]]                                   # Full generated narration/script
    slide_segments: List[Dict[str, str]]             # Segmented slide content

In [109]:

def generate_subtopics(state: AgentState) -> AgentState:
    """
    Generate slide-worthy subtopics from a given user prompt.

    This tool asks the LLM to break the provided topic into 3-4 concise 
    and informative subtopics suitable for video slides, without considering 
    any duration specified by the user.

    Args:
        user_prompt (str): The user's input prompt containing the topic.

    Returns:
        list[str]: A list of concise subtopics suitable for video slides.
    """
    prompt = state['user_prompt']
    response = llm.invoke(f"Given the topic: '{prompt}', generate ONLY 5-7 concise subtopic titles "
        "as a numbered list (1. Title, 2. Title, ...). "
        "Do NOT include any explanations, definitions, or extra details.")
    state['subtopics'] = [s.strip() for s in response.content.split("\n") if s.strip()]
    return state

In [110]:
load_dotenv()
LANGSEARCH_API_KEY = os.getenv("LANGSEARCH_API_KEY")
LANGSEARCH_API_URL = "https://api.langsearch.com/v1/web-search"
import requests


def fetch_resources(state: AgentState) -> AgentState:
    """
    Fetch web resources for each subtopic using LangSearch API.

    Args:
        state (AgentState): The current agent state with subtopics.

    Returns:
        AgentState: Updated state with scraped_data populated.
    """
    headers = {
        "Authorization": f"Bearer {LANGSEARCH_API_KEY}",
        "Content-Type": "application/json"
    }

    for topic in state["subtopics"]:
        payload = {
            "query": topic,
            "freshness": "noLimit",
            "summary": True,
            "count": 2
        }

        response = requests.post(LANGSEARCH_API_URL, headers=headers, json=payload)

        topic_results = []
        if response.status_code == 200:
            data = response.json()
            web_pages = data.get("data", {}).get("webPages", {}).get("value", [])

            for page in web_pages:
                topic_results.append({
                    "title": page.get("name"),
                    "url": page.get("url"),
                    "summary": page.get("summary")
                })

        state["scraped_data"].append({
            "topic": topic,
            "results": topic_results
        })

    return state


In [118]:
import re

def create_script(state: AgentState) -> AgentState:
    """
    For each subtopic, combine scraped content and generate:
    - Key facts for slides.
    - Narration script for presenters.

    Updates:
    - state["full_script"]: List of dicts per subtopic with 'topic', 'narration', 'facts'.

    Args:
        state (AgentState): The agent state with scraped_data.

    Returns:
        AgentState: Updated with structured full_script.
    """
    state["full_script"] = []

    for data in state["scraped_data"]:
        topic = data.get("topic", "")
        results = data.get("results", [])

        combined_summary = "\n".join(
            result.get("summary", "") for result in results if result.get("summary", "")
        )
        if len(combined_summary) > 4000:
            combined_summary = combined_summary[:4000]

        if not combined_summary:
            continue

        llm_prompt = (
            f"Using the following information as context:\n'''{combined_summary}'''\n\n"
            f"Generate the following two outputs strictly in PLAIN TEXT (no HTML, no markdown, no special formatting). "
            f"Use EXACTLY the headings 'Facts:' and 'Narration Script:' with no extra symbols or decoration.\n\n"

            f"Facts:\n"
            f"List exactly 2 to 3 key facts about the topic '{topic}'. Format them as:\n"
            f"1. First fact\n"
            f"2. Second fact\n"
            f"3. Third fact (optional)\n"
            f"Do NOT use bullet points, dashes, asterisks, markdown symbols, or any formatting — ONLY numbered plain text facts.\n\n"

            f"Narration Script:\n"
            f"Write a complete, engaging YouTube video narration script for the topic '{topic}'. "
            f"The narration MUST explain the facts listed above in a natural, conversational, and friendly tone, "
            f"as if the presenter is talking directly to the audience. "
            f"Do NOT include any stage directions, sound effects, descriptions of background music, presenter actions, or visual cues. "
            f"Ensure this is strictly plain text with no HTML, markdown, or any other formatting.\n"
        )


        response = llm.invoke(llm_prompt)
        output = response.content.strip()

        # Split based on the "2. Narration Script" section, case-insensitive
        split_match = re.split(r"\bNarration Script:\s*", output, maxsplit=1, flags=re.IGNORECASE)
        facts_part = split_match[0].replace("Facts:", "").strip()
        narration_part = split_match[1].strip() if len(split_match) > 1 else ""


        state["full_script"].append({
            "topic": topic,
            "facts": facts_part,
            "narration": narration_part
        })

    return state


In [112]:
import re

def create_slide_segments(state: AgentState) -> AgentState:
    """
    Generates structured slide segments for each subtopic based on narration and facts.
    
    For each slide:
    - content_to_display: Contains the specific facts relevant to that slide.
    - narration_script: Explains the facts shown in content_to_display.
    - is_blank_slide: True if the slide is meant to show stock video only.
    """
    state["slide_segments"] = []
    slide_counter = 1

    for script_data in state["full_script"]:
        subtopic = script_data["topic"]
        narration = script_data["narration"]
        facts = script_data["facts"]

        llm_prompt = (
            f"For the subtopic '{subtopic}', you are provided with:\n\n"
            f"Facts:\n{facts}\n\n"
            f"Narration Script:\n{narration}\n\n"
            f"Your task is to divide this content into up to 2 slides. For each slide:\n"
            f"- In 'Display', ONLY show the relevant facts for that slide.\n"
            f"- In 'Narration', provide the portion of the narration that directly explains the displayed facts.\n"
            f"- If there's a narration section that doesn't require any fact display (like general commentary), mark that slide's display as 'BLANK_SLIDE'.\n\n"
            f"STRICT RULES:\n"
            f"- Maximum of 2 slides per subtopic.\n"
            f"- Only split into 2 if necessary.\n"
            f"- Always pair facts with their explanations.\n\n"
            f"Format your output strictly as:\n"
            f"Slide 1:\nDisplay: <Facts or BLANK_SLIDE>\nNarration: <Narration for this slide>\n\n"
            f"Slide 2:\nDisplay: <Facts or BLANK_SLIDE>\nNarration: <Narration for this slide>\n\n"
            f"Ensure clarity and direct fact-to-narration mapping."
        )

        response = llm.invoke(llm_prompt)
        slides_text = response.content.strip()

        # Parse LLM response into slides
        slide_matches = re.split(r'Slide \d+:', slides_text)
        for slide_data in slide_matches:
            if not slide_data.strip():
                continue

            display_match = re.search(r'Display:\s*(.*?)\nNarration:', slide_data, re.DOTALL)
            narration_match = re.search(r'Narration:\s*(.*)', slide_data, re.DOTALL)

            content_to_display = display_match.group(1).strip() if display_match else ""
            narration_text = narration_match.group(1).strip() if narration_match else ""

            is_blank_slide = content_to_display.strip().upper() == "BLANK_SLIDE"

            state["slide_segments"].append({
                "slide_no": slide_counter,
                "subtopic": subtopic,
                "content_to_display": content_to_display,
                "narration_script": narration_text,
                "is_blank_slide": is_blank_slide
            })

            slide_counter += 1

    return state


In [113]:
graph_builder = StateGraph(AgentState)

graph_builder.add_node("Generate_Subtopics", generate_subtopics)
graph_builder.add_node("Fetch_Resources", fetch_resources)
graph_builder.add_node("Create_Script", create_script)
graph_builder.add_node("Create_Slide_Segments", create_slide_segments)

In [114]:
graph_builder.add_edge(START, "Generate_Subtopics")
graph_builder.add_edge("Generate_Subtopics", "Fetch_Resources")
graph_builder.add_edge("Fetch_Resources", "Create_Script")
graph_builder.add_edge("Create_Script", "Create_Slide_Segments")
graph_builder.add_edge("Create_Slide_Segments", END)

graph = graph_builder.compile()


In [119]:
user_prompt = input("Enter your topic prompt: ")

initial_state = {
    "messages": [],
    "user_prompt": user_prompt,
    "scraped_data": [],
    "subtopics": [],
    "full_script": [],
    "slide_segments": []
}

final_state = graph.invoke(initial_state)

from pprint import pprint

print("\n--- Subtopics ---")
pprint(final_state["subtopics"])

print("\n--- Full Script ---")
pprint(final_state["full_script"])

print("\n--- Slide Segments ---")
pprint(final_state["slide_segments"])


--- Subtopics ---
['1. Popular Dishes',
 '2. Regional Cuisine',
 '3. Spices and Seasonings',
 '4. Street Food',
 '5. Desserts and Sweets',
 '6. Cooking Techniques',
 '7. Traditional Festivals']

--- Full Script ---
[{'facts': 'Facts:\n'
           '1. The most popular dishes in Europe are pizza, ramen, burger, '
           'paella, moussaka, boeuf bourguignon, pierogi, tikka massala, '
           "eisbein, and tom kha gai, according to Photobox's study.\n"
           '2. Italian, Japanese, and American foods are the most popular in '
           'Europe, accounting for the majority of the top 10 dishes.\n'
           '3. The study found that food interacts with memory recall, with '
           'experiences like baking a certain type of biscuit with children '
           'creating memories that can be easily recalled later.\n'
           '\n'
           'Narration Script:\n'
           "Welcome to our culinary journey around the world! Today, we're "
           'going to explore the mos